In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/HamKalleh/CV/OCR/Segmentation_Models/DBNet

/content/drive/MyDrive/HamKalleh/CV/OCR/Segmentation_Models/DBNet


#Requirements

In [3]:
!pip install pyclipper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 11.7 MB/s eta 0:00:00


#Libraries

In [4]:
import os
import copy
import cv2
import numpy
import torch
import tqdm
import math
import csv
import random
import warnings
from nets import nn
from utils import util
from torch.utils import data
from utils.dataset import Dataset
from google.colab.patches import cv2_imshow

warnings.filterwarnings("ignore")

#Train

In [ ]:
import math
import random
import cv2
import numpy
import numpy as np
import torch
from pyclipper import *
from shapely import Polygon
from torch.utils import data

def resample():
    choices = (cv2.INTER_AREA,
               cv2.INTER_CUBIC,
               cv2.INTER_LINEAR,
               cv2.INTER_NEAREST,
               cv2.INTER_LANCZOS4)
    return random.choice(seq=choices)


def resize(image, input_size):
    shape = image.shape
    width = shape[1] * input_size / shape[0]
    width = math.ceil(width / 32) * 32
    return cv2.resize(image, dsize=(width, input_size))


class RandomAffine:
    def __init__(self):
        self.params = {'degrees': 15,
                       'scale': 0.50,
                       'shear': 0.001,
                       'translate': 0.25}

    def __call__(self, image, b, b_mask, t, t_mask):
        h, w = image.shape[:2]

        # Center
        center = numpy.eye(3)
        center[0, 2] = -image.shape[1] / 2  # x translation (pixels)
        center[1, 2] = -image.shape[0] / 2  # y translation (pixels)

        # Perspective
        perspective = numpy.eye(3)

        # Rotation and Scale
        rotate = numpy.eye(3)
        a = random.uniform(-self.params['degrees'], self.params['degrees'])
        s = random.uniform(1 - self.params['scale'], 1 + self.params['scale'])
        rotate[:2] = cv2.getRotationMatrix2D(angle=a, center=(0, 0), scale=s)

        # Shear
        shear = numpy.eye(3)
        shear[0, 1] = math.tan(random.uniform(-self.params['shear'], self.params['shear']) * math.pi / 180)
        shear[1, 0] = math.tan(random.uniform(-self.params['shear'], self.params['shear']) * math.pi / 180)

        # Translation
        translate = numpy.eye(3)
        translate[0, 2] = random.uniform(0.5 - self.params['translate'], 0.5 + self.params['translate']) * w
        translate[1, 2] = random.uniform(0.5 - self.params['translate'], 0.5 + self.params['translate']) * h

        # Combined rotation matrix, order of operations (right to left) is IMPORTANT
        matrix = translate @ shear @ rotate @ perspective @ center
        if numpy.any((matrix != numpy.eye(3))):  # image changed
            image = cv2.warpAffine(image, matrix[:2], dsize=(w, h), borderValue=(0, 0, 0))
            b = cv2.warpAffine(b, matrix[:2], dsize=(w, h), borderValue=(0, 0, 0))
            t = cv2.warpAffine(t, matrix[:2], dsize=(w, h), borderValue=(0, 0, 0))
            b_mask = cv2.warpAffine(b_mask, matrix[:2], dsize=(w, h), borderValue=(1, 1, 1))
            t_mask = cv2.warpAffine(t_mask, matrix[:2], dsize=(w, h), borderValue=(0, 0, 0))
        return image, b, b_mask, t, t_mask


class RandomHSV:
    def __init__(self,
                 h_delta: int = 5,
                 s_delta: int = 30,
                 v_delta: int = 30):
        self.h_delta = h_delta
        self.s_delta = s_delta
        self.v_delta = v_delta

    def random_hsv(self):
        h = self.h_delta
        s = self.s_delta
        v = self.v_delta
        hsv_gains = numpy.random.uniform(-1, 1, 3) * [h, s, v]
        # random selection of h, s, v
        hsv_gains *= numpy.random.randint(0, 2, 3)
        # prevent overflow
        hsv_gains = hsv_gains.astype(numpy.int16)
        return hsv_gains

    def __call__(self, img):
        hsv = self.random_hsv()
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(numpy.int16)

        img_hsv[..., 0] = (img_hsv[..., 0] + hsv[0]) % 180
        img_hsv[..., 1] = numpy.clip(img_hsv[..., 1] + hsv[1], 0, 255)
        img_hsv[..., 2] = numpy.clip(img_hsv[..., 2] + hsv[2], 0, 255)
        return cv2.cvtColor(img_hsv.astype(img.dtype), cv2.COLOR_HSV2BGR)

class RandomResize:
    def __init__(self, size):
        self.size = size
        self.scale = (size, size)
        self.ratio_range = (0.1, 2.0)

    def resize(self, x, b, b_mask, t, t_mask):
        min_ratio, max_ratio = self.ratio_range
        assert min_ratio <= max_ratio
        ratio = numpy.random.random_sample()
        ratio = ratio * (max_ratio - min_ratio) + min_ratio
        scale = int(self.scale[0] * ratio), int(self.scale[1] * ratio)

        shape = x.shape[:2]
        scale = min(max(scale) / max(shape), min(scale) / min(shape))

        w = int(shape[1] * scale + 0.5)
        h = int(shape[0] * scale + 0.5)

        x = cv2.resize(x, dsize=(w, h), interpolation=resample())
        b = cv2.resize(b, dsize=(w, h), interpolation=cv2.INTER_NEAREST)
        t = cv2.resize(t, dsize=(w, h), interpolation=cv2.INTER_NEAREST)
        b_mask = cv2.resize(b_mask, dsize=(w, h), interpolation=cv2.INTER_NEAREST)
        t_mask = cv2.resize(t_mask, dsize=(w, h), interpolation=cv2.INTER_NEAREST)
        return x, b, b_mask, t, t_mask

    def pad(self, x, b, b_mask, t, t_mask):
        shape = (x.shape[1], x.shape[0])
        ratio = float(self.size) / max(shape)
        size = tuple([int(x * ratio) for x in shape])

        if size[0] > self.size or size[1] > self.size:
            ratio = float(self.size) / min(shape)
            size = tuple([int(x * ratio) for x in shape])

        w = self.size - size[0]
        h = self.size - size[1]
        top, bottom = h // 2, h - (h // 2)
        left, right = w // 2, w - (w // 2)
        x = cv2.resize(x, size, interpolation=resample())
        b = cv2.resize(b, size, interpolation=cv2.INTER_NEAREST)
        t = cv2.resize(t, size, interpolation=cv2.INTER_NEAREST)
        b_mask = cv2.resize(b_mask, size, interpolation=cv2.INTER_NEAREST)
        t_mask = cv2.resize(t_mask, size, interpolation=cv2.INTER_NEAREST)
        x = cv2.copyMakeBorder(x, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        b = cv2.copyMakeBorder(b, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        t = cv2.copyMakeBorder(t, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        b_mask = cv2.copyMakeBorder(b_mask, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(1, 1, 1))
        t_mask = cv2.copyMakeBorder(t_mask, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        return x, b, b_mask, t, t_mask

    def __call__(self, x, b, b_mask, t, t_mask):
        x, b, b_mask, t, t_mask = self.resize(x, b, b_mask, t, t_mask)
        x, b, b_mask, t, t_mask = self.pad(x, b, b_mask, t, t_mask)
        return x, b, b_mask, t, t_mask

class Transform:
    def __init__(self):
        self.shrink_ratio = .4
        self.min_text_size = 8
        self.t_min = 0.3
        self.t_max = 0.7

    def __call__(self, image, polygons, ignore_tags):
        h, w = image.shape[:2]

        b = numpy.zeros(shape=(h, w), dtype=numpy.float32)
        b_mask = numpy.ones(shape=(h, w), dtype=numpy.float32)

        polygons, ignore_tags = self.validate_polygons(polygons, ignore_tags, h, w)
        for i, polygon in enumerate(polygons):
            height = max(polygon[:, 1]) - min(polygon[:, 1])
            width = max(polygon[:, 0]) - min(polygon[:, 0])
            if ignore_tags[i] or min(height, width) < self.min_text_size:
                cv2.fillPoly(b_mask, polygon.astype(numpy.int32)[numpy.newaxis, :, :], 0)
                ignore_tags[i] = True
            else:
                polygon_shape = Polygon(polygon)
                distance = polygon_shape.area * (1 - numpy.power(self.shrink_ratio, 2)) / polygon_shape.length
                padding = PyclipperOffset()
                padding.AddPath([tuple(p) for p in polygons[i]], JT_ROUND, ET_CLOSEDPOLYGON)
                padded = padding.Execute(-distance)
                if not padded:
                    cv2.fillPoly(b_mask, polygon.astype(numpy.int32)[numpy.newaxis, :, :], 0)
                    ignore_tags[i] = True
                    continue
                padded = numpy.array(padded[0]).reshape(-1, 2)
                cv2.fillPoly(b, [padded.astype(numpy.int32)], 1)

        t = numpy.zeros(image.shape[:2], dtype=numpy.float32)
        t_mask = numpy.zeros(image.shape[:2], dtype=numpy.float32)
        for i in range(len(polygons)):
            if ignore_tags[i]:
                continue
            self.draw_border_map(polygons[i], t, mask=t_mask)
        t = t * (self.t_max - self.t_min) + self.t_min
        return image, b, b_mask, t, t_mask

    def validate_polygons(self, polygons, ignore_tags, h, w):
        if len(polygons) == 0:
            return polygons, ignore_tags
        assert len(polygons) == len(ignore_tags)
        for polygon in polygons:
            polygon[:, 0] = numpy.clip(polygon[:, 0], 0, w - 1)
            polygon[:, 1] = numpy.clip(polygon[:, 1], 0, h - 1)

        for i in range(len(polygons)):
            area = self.polygon_area(polygons[i])
            if abs(area) < 1:
                ignore_tags[i] = True
            if area > 0:
                polygons[i] = polygons[i][::-1, :]
        return polygons, ignore_tags

    @staticmethod
    def polygon_area(polygon):
        edge = 0
        for i in range(polygon.shape[0]):
            next_index = (i + 1) % polygon.shape[0]
            edge += (polygon[next_index, 0] - polygon[i, 0]) * (polygon[next_index, 1] + polygon[i, 1])

        return edge / 2.

    def draw_border_map(self, polygon, canvas, mask):
        polygon = numpy.array(polygon)

        assert polygon.ndim == 2
        assert polygon.shape[1] == 2

        polygon_shape = Polygon(polygon)
        distance = polygon_shape.area * (1 - numpy.power(self.shrink_ratio, 2)) / polygon_shape.length
        subject = [tuple(p) for p in polygon]
        padding = PyclipperOffset()
        padding.AddPath(subject, JT_ROUND, ET_CLOSEDPOLYGON)
        padded_polygon = numpy.array(padding.Execute(distance)[0])
        cv2.fillPoly(mask, [padded_polygon.astype(numpy.int32)], 1.0)

        x1 = padded_polygon[:, 0].min()
        x2 = padded_polygon[:, 0].max()
        y1 = padded_polygon[:, 1].min()
        y2 = padded_polygon[:, 1].max()

        w = x2 - x1 + 1
        h = y2 - y1 + 1

        polygon[:, 0] = polygon[:, 0] - x1
        polygon[:, 1] = polygon[:, 1] - y1

        xs = numpy.broadcast_to(numpy.linspace(0, w - 1, num=w).reshape(1, w), (h, w))
        ys = numpy.broadcast_to(numpy.linspace(0, h - 1, num=h).reshape(h, 1), (h, w))

        distance_map = numpy.zeros((polygon.shape[0], h, w), dtype=numpy.float32)
        for i in range(polygon.shape[0]):
            j = (i + 1) % polygon.shape[0]
            absolute_distance = self.distance(xs, ys, polygon[i], polygon[j])
            distance_map[i] = numpy.clip(absolute_distance / distance, 0, 1)
        distance_map = distance_map.min(axis=0)

        x1_valid = min(max(0, x1), canvas.shape[1] - 1)
        x2_valid = min(max(0, x2), canvas.shape[1] - 1)
        y1_valid = min(max(0, y1), canvas.shape[0] - 1)
        y2_valid = min(max(0, y2), canvas.shape[0] - 1)

        canvas_valid = canvas[y1_valid:y2_valid + 1, x1_valid:x2_valid + 1]
        distance_map_valid = distance_map[y1_valid - y1:y2_valid - y2 + h, x1_valid - x1:x2_valid - x2 + w]

        canvas[y1_valid:y2_valid + 1, x1_valid:x2_valid + 1] = numpy.fmax(1 - distance_map_valid, canvas_valid)

    @staticmethod
    def distance(xs, ys, point_1, point_2):
        square_distance_1 = numpy.square(xs - point_1[0]) + numpy.square(ys - point_1[1])
        square_distance_2 = numpy.square(xs - point_2[0]) + numpy.square(ys - point_2[1])
        square_distance = numpy.square(point_1[0] - point_2[0]) + numpy.square(point_1[1] - point_2[1])

        cosine = (square_distance - square_distance_1 - square_distance_2) / \
                 (2 * numpy.sqrt(square_distance_1 * square_distance_2))
        square_sin = 1 - numpy.square(cosine)
        square_sin = numpy.nan_to_num(square_sin)

        result = numpy.sqrt(square_distance_1 * square_distance_2 * square_sin / square_distance)
        result[cosine < 0] = numpy.sqrt(numpy.fmin(square_distance_1, square_distance_2))[cosine < 0]
        return result

class Dataset(data.Dataset):
    def __init__(self, filenames, input_size, train):
        labels = self.load_labels(filenames, train)
        self.train = train
        self.labels = list(labels.values())
        self.filenames = list(labels) #.keys())
        self.input_size = input_size

        self.transform = Transform()
        self.random_hsv = RandomHSV()
        self.random_affine = RandomAffine()
        self.random_resize = RandomResize(input_size)

        self.mean = numpy.array([0.406, 0.456, 0.485]).reshape((1, 1, 3)).astype('float32')
        self.std = numpy.array([0.225, 0.224, 0.229]).reshape((1, 1, 3)).astype('float32')

    def __getitem__(self, index):
        image = cv2.imread(self.filenames[index], cv2.IMREAD_COLOR).astype('float32')
        label = self.labels[index]

        H, W = image.shape[:2]

        shape = image.shape

        annotations = []
        for line in label:
            annotations.append({'points': [(p[0], p[1]) for p in line['poly']],
                                'ignore': line['text'] == '###',
                                'text': line['text']})
        polygons = []
        ignore_tags = []

        for annotation in annotations:
            polygons.append(numpy.array(annotation['points']))
            ignore_tags.append(annotation['ignore'])
        ignore_tags = numpy.array(ignore_tags, dtype=numpy.uint8)

        if self.train:
            image = self.apply_minimal_augmentations(image)
            image, b, b_mask, t, t_mask = self.transform(image, polygons, ignore_tags)
            image, b, b_mask, t, t_mask = self.random_resize(image, b, b_mask, t, t_mask)
            image, b, b_mask, t, t_mask = self.random_affine(image, b, b_mask, t, t_mask)
            image, b, b_mask, t, t_mask = self.flip_lr(image, b, b_mask, t, t_mask)

            image = self.random_hsv(image)
            image = self.to_tensor(image)

            return image, {'b': b[numpy.newaxis, :, :], 'b_mask': b_mask, 't': t, 't_mask': t_mask}
        else:
            image = resize(image, self.input_size)
            image = self.to_tensor(image)

            return image, tuple(shape[:2]), polygons, ignore_tags

    def apply_minimal_augmentations(self, image):
        # Convert to HSV for hue manipulation
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Slight hue adjustment (±10 degrees)
        hue_shift = random.uniform(-12, 12)
        hsv[:, :, 0] = (hsv[:, :, 0] + hue_shift) % 180

        # Convert back to BGR
        image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # Slight brightness adjustment (±10%)
        brightness = random.uniform(0.7, 1.3)
        image = image * brightness

        # Slight contrast adjustment (±10%)
        contrast = random.uniform(0.7, 1.3)
        mean_val = np.mean(image)
        image = contrast * (image - mean_val) + mean_val

        # Ensure values stay within valid range
        image = np.clip(image, 0, 255)

        return image

    def __len__(self):
        return len(self.filenames)

    def to_tensor(self, image):
        image = image.astype('float32') / 255.0
        image = image - self.mean
        image = image / self.std

        image = image.transpose((2, 0, 1))[::-1]
        image = numpy.ascontiguousarray(image)
        image = torch.from_numpy(image)
        return image

    @staticmethod
    def flip_lr(image, b, b_mask, t, t_mask):
        # Flip left-right
        if random.random() < 0.5:
            image = cv2.flip(image, flipCode=1)
            b = cv2.flip(b, flipCode=1)
            t = cv2.flip(t, flipCode=1)
            b_mask = cv2.flip(b_mask, flipCode=1)
            t_mask = cv2.flip(t_mask, flipCode=1)
        return image, b, b_mask, t, t_mask

    @staticmethod
    def load_labels(filenames, train):
        labels = {}
        for filename in filenames:
            lines = []

            if train:
              path = "Dataset/TotalText/labels_aug/"
            else:
              path = "Dataset/TotalText/labels_aug/"

            with open(path + filename.split('/')[-1].split('.')[0] + '.txt', 'r') as f:
                for line in f.readlines():
                    item = {}
                    parts = line.strip().split(',')
                    line = [i.strip('\ufeff').strip('\xef\xbb\xbf') for i in parts]
                    num_points = math.floor((len(line) - 1) / 2) * 2
                    poly = numpy.array(list(map(float, line[:num_points]))).reshape((-1, 2)).tolist()
                    item['poly'] = poly
                    item['text'] = parts[-1]
                    lines.append(item)
            labels[filename] = lines
        return labels

    @staticmethod
    def collate_fn(batch):
        image, shape, polygons, ignore_tags = zip(*batch)
        samples = torch.stack(tensors=image, dim=0)
        targets = {'shape': shape,
                   'polygons': polygons,
                   'ignore_tags': ignore_tags}

        return samples, targets

In [ ]:
def lr(batch_size, world_size):
    return 1E-2 * batch_size * world_size / 1

def resume_from_checkpoint(model, optimizer, scheduler, scaler, path):
    checkpoint = torch.load(path, map_location='cuda')
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])
    scaler.load_state_dict(checkpoint['scaler'])
    return checkpoint['epoch']

def train(input_shape, batch_size, local_rank, world_size, distributed, epochs, train_filenames, test_filenames):

  # Model
  # model = nn.DBNet()
  # model = util.load_checkpoint(model, ckpt='./weights/rotate_best.pt')#'./weights/imagenet.pt')
  # model.cuda()

  model = nn.DBNet()
  checkpoint = torch.load('./weights/db_best.pt')#, map_location='cpu')
  model.load_state_dict(checkpoint['model'])
  model.float().cuda()

  # Optimizer
  optimizer = torch.optim.SGD(util.weight_decay(model), lr(batch_size, world_size), momentum=0.9, nesterov=True)
  # EMA
  ema = util.EMA(model) if local_rank == 0 else None
  dataset = Dataset(train_filenames, input_shape, train=True)
  sampler = data.distributed.DistributedSampler(dataset) if distributed else None
  loader = data.DataLoader(dataset, batch_size, sampler is None,
                         sampler=sampler, num_workers=8, pin_memory=True)

  # DDP
  if distributed:
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
        model = torch.nn.parallel.DistributedDataParallel(module=model,
                                                          device_ids=[local_rank],
                                                          output_device=local_rank)

  # Loss, AMP, Scheduler
  criterion = util.ComputeLoss().cuda()
  amp_scale = torch.cuda.amp.GradScaler()
  #scheduler = util.CosineLR(epochs, optimizer)
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

  # Try resuming
  # resume_path = './weights/kale_last_final.pt'
  start_epoch = 0
  # if os.path.exists(resume_path):
  #       start_epoch = resume_from_checkpoint(model, optimizer, scheduler, amp_scale, resume_path) + 1
  #       print(f"Resumed from checkpoint at epoch {start_epoch}")

  best = 0
  num_steps = len(loader)

  if local_rank == 0:
        os.makedirs('weights', exist_ok=True)
        log_file = open('weights/step.csv', 'a' if start_epoch > 0 else 'w')
        logger = csv.DictWriter(log_file, fieldnames=['epoch', 'loss', 'Recall', 'Precision', 'F1'])
        if start_epoch == 0:
            logger.writeheader()

  for epoch in range(start_epoch, epochs):
        model.train()
        if distributed:
            sampler.set_epoch(epoch)

        p_bar = tqdm.tqdm(loader, total=num_steps) if local_rank == 0 else loader
        avg_loss = util.AverageMeter()

        if local_rank == 0:
            print(('\n' + '%10s' * 3) % ('epoch', 'memory', 'loss'))

        optimizer.zero_grad()
        for samples, targets in p_bar:
            samples = samples.cuda()
            with torch.cuda.amp.autocast():
                outputs = model(samples)
                loss = criterion(outputs, targets)

            amp_scale.scale(loss).backward()
            amp_scale.step(optimizer)
            amp_scale.update()
            optimizer.zero_grad()

            if ema:
                ema.update(model)

            # Log
            if distributed:
                loss = utils.reduce_tensor(loss.data, world_size)
            avg_loss.update(loss.item(), samples.size(0))
            if local_rank == 0:
                memory = f'{torch.cuda.memory_reserved() / 1E9:.3g}G'
                s = ('%10s' * 2 + '%10.3g') % (f'{epoch + 1}/{epochs}', memory, avg_loss.avg)
                p_bar.set_description(s)

        # Scheduler step
        scheduler.step() #epoch, optimizer)

        if local_rank == 0:
            last = test(test_filenames, ema.ema)
            logger.writerow({
                'epoch': str(epoch + 1).zfill(3),
                'loss': f'{avg_loss.avg:.3f}',
                'Recall': f'{last[1]:.3f}',
                'Precision': f'{last[0]:.3f}',
                'F1': f'{last[2]:.3f}'
            })
            log_file.flush()

            # Save checkpoint
            is_best = last[2] > best
            if is_best:
                best = last[2]

            checkpoint = {
                'epoch': epoch,
                'model': ema.ema.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),
                'scaler': amp_scale.state_dict()
            }

            torch.save(checkpoint, './weights/db_last.pt') #final.pt')
            if is_best:
                torch.save(checkpoint, './weights/db_best.pt') #final.pt')
                print("best weights so far!")

  if local_rank == 0:
        log_file.close()
        # util.strip_optimizer('./weights/kale_best_final.pt')
        # util.strip_optimizer('./weights/kale_last_final.pt')

  torch.cuda.empty_cache()

In [ ]:
input_shape = 800
batch_size = 16
local_rank = 0
world_size = int(os.getenv('WORLD_SIZE', 1))
distributed = int(os.getenv('WORLD_SIZE', 1)) > 1
epochs = 500

train_filenames = []
with open('Dataset/train.txt') as f:
  for filename in f.readlines():
    filename = filename.rstrip()
    train_filenames.append('Dataset/train/images/' + filename)

test_filenames = []
with open('Dataset/val.txt') as f:
  for filename in f.readlines():
    filename = filename.rstrip()
    test_filenames.append('Dataset/val/images/' + filename)

train(input_shape,
      batch_size,
      local_rank,
      world_size,
      distributed,
      epochs,
      train_filenames,
      test_filenames[:3])

  0%|          | 0/7 [00:00<?, ?it/s]


     epoch    memory      loss


 precision    recall        F1: 100%|██████████| 3/3 [00:00<00:00,  9.65it/s]


     0.333     0.571     0.421
best weights so far!


  0%|          | 0/7 [00:00<?, ?it/s]

...


     epoch    memory      loss


 precision    recall        F1: 100%|██████████| 3/3 [00:00<00:00,  9.37it/s]


     0.483     1.000     0.651


  0%|          | 0/7 [00:00<?, ?it/s]


     epoch    memory      loss


#Test

In [ ]:
@torch.no_grad()
def test(filenames, model=None):

    input_shape = 800

    dataset = Dataset(filenames, input_shape, train=False)
    loader = data.DataLoader(dataset, collate_fn=Dataset.collate_fn)

    if model is None:
      model = nn.DBNet()
      checkpoint = torch.load('./weights/db_best.pt')#, map_location='cpu')
      model.load_state_dict(checkpoint['model'])
      model.float().cuda()

    model.eval()

    results = []

    evaluator = util.QuadMeasurer(is_polygon=True)
    for sample, target in tqdm.tqdm(loader, ('%10s' * 3) % ('precision', 'recall', 'F1')):
        output = model(sample.cuda())

        output = util.mask_to_box(target, output.cpu(), is_polygon=True)

        result = evaluator.validate_measure(target, output)
        results.append(result)

    precision, recall, f1 = evaluator.gather_measure(results)
    # Print results
    print(('%10s' * 3) % (f'{precision:.3f}', f'{recall:.3f}', f'{f1:.3f}'))

    # Return results
    model.float()  # for training
    return precision, recall, f1

In [ ]:
test_filenames = []
with open('Dataset/test.txt') as f:
  for filename in f.readlines():
    filename = filename.rstrip()
    test_filenames.append('Dataset/test/images/' + filename)

test(test_filenames)

 precision    recall        F1: 100%|██████████| 25/25 [00:26<00:00,  1.05s/it]

     0.996     0.736     0.846


(0.9955947136563875, 0.7361563517915308, 0.8464419426777624)

#Demo

In [ ]:
import numpy
import cv2
import random
import math
import time
from google.colab.patches import cv2_imshow

@torch.no_grad()
def demo(filenames, model=None):
    input_size = 800

    model = nn.DBNet()
    checkpoint = torch.load('./weights/db_last.pt', map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    model.float()#.cuda()

    model.eval()

    mean = numpy.array([0.406, 0.456, 0.485]).reshape((1, 1, 3)).astype('float32')
    std = numpy.array([0.225, 0.224, 0.229]).reshape((1, 1, 3)).astype('float32')
    for filename in tqdm.tqdm(filenames):
        image = cv2.imread(filename, cv2.IMREAD_COLOR)
        print(image.shape)

        shape = image.shape[:2]

        width = shape[1] * input_size / shape[0]
        width = math.ceil(width / 32) * 32

        x = cv2.resize(image, dsize=(width, input_size))
        x = x.astype('float32') / 255.0
        x = x - mean
        x = x / std
        x = x.transpose((2, 0, 1))[::-1]
        x = numpy.ascontiguousarray(x)
        x = torch.from_numpy(x)
        x = x.unsqueeze(0)
        #x = x.cuda()

        init = time.time()
        output = model(x)

        output = util.mask_to_box(targets={'shape': [shape]}, outputs=output.cpu(), is_polygon=True)

        print(time.time()-init)
        boxes, scores = output[0][0], output[1][0]

        for box in boxes:
            box = numpy.array(box).reshape((-1, 1, 2))
            cv2.polylines(image, [box], isClosed=True, color=(0, 255, 0), thickness=5)

        cv2_imshow(image)
        #cv2.imwrite("/content/DBNet/"+filename.split('/')[-1], image) #f'./data/{os.path.basename(filename)}', image)

filenames = []
with open('Dataset/test/test_aug.txt') as f:
  for filename in f.readlines():
    filename = filename.rstrip()
    filenames.append('Dataset/test/images_aug/' + filename)
demo(filenames)

In [ ]:
import numpy
import cv2
import random
import math
import time
from google.colab.patches import cv2_imshow

@torch.no_grad()
def demo(filename, model=None):
    input_size = 800

    model = nn.DBNet()
    checkpoint = torch.load('./weights/db_last.pt', map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    model.float()#.cuda()

    model.eval()

    mean = numpy.array([0.406, 0.456, 0.485]).reshape((1, 1, 3)).astype('float32')
    std = numpy.array([0.225, 0.224, 0.229]).reshape((1, 1, 3)).astype('float32')

    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    #image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    image = cv2.rotate(image, cv2.ROTATE_180)
    # image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    print(image.shape)

    shape = image.shape[:2]

    width = shape[1] * input_size / shape[0]
    width = math.ceil(width / 32) * 32

    x = cv2.resize(image, dsize=(width, input_size))
    x = x.astype('float32') / 255.0
    x = x - mean
    x = x / std
    x = x.transpose((2, 0, 1))[::-1]
    x = numpy.ascontiguousarray(x)
    x = torch.from_numpy(x)
    x = x.unsqueeze(0)
    #x = x.cuda()

    init = time.time()
    output = model(x)

    output = util.mask_to_box(targets={'shape': [shape]}, outputs=output.cpu(), is_polygon=True)

    print(time.time()-init)
    boxes, scores = output[0][0], output[1][0]

    for box in boxes:
        box = numpy.array(box).reshape((-1, 1, 2))
        cv2.polylines(image, [box], isClosed=True, color=(0, 255, 0), thickness=5)

    cv2_imshow(image)
    #cv2.imwrite("/content/DBNet/"+filename.split('/')[-1], image) #f'./data/{os.path.basename(filename)}', image)

demo("/content/2b2832cb-d805-4d5e-b10c-85bf5a94cebc.jpg")